In [ ]:
# TODO: test that ps3.get_pdf_text(asset_url) is working as expected.


In [ ]:
# !pip uninstall -y foresight
# !pip install git+https://github.com/SAGE-3/next.git@dev#subdirectory=foresight
# !pip install langchain_openai
# !pip install langchain

In [1]:
from langchain.globals import set_debug
set_debug(True)


In [2]:
from langgraph.prebuilt import tools_condition

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langgraph.prebuilt import ToolNode

import sys
sys.path.append("/Users/mahdi/Documents/GitHub/next/foresight/")
# from src.config import config as conf, prod_type
import copy
from foresight.Sage3Sugar.pysage3 import PySage3
from src.prompts.prompt_loader import PromptLoader 
from langchain_openai import ChatOpenAI
keys_to_remove = ['_id', '_createdAt', '_createdBy', '_updatedAt', '_updatedBy',
    'touched', 'executeInfo', 'path', 'raised', 
     'roomId', 'boardId', 'lock', 'dragging', 'pinned', 'rotation'
]

PROMPTS_DIR = "src/prompts/"

In [4]:
prod_type= 'local'
conf = {'local': {'seer_server': 'http://127.0.0.1:5002',
  'jupyter_server': 'http://localhost:8888',
  'redis_server': 'localhost',
  'web_server': 'http://localhost:3333',
  'ws_server': 'ws://localhost:3333'}}
room_id = "d6640933-7b9f-4536-a263-02a514ff092a"
board_id = "02fc98df-b4f1-470c-8516-7221c3095de7"

# app_id = '4a29b8bf-9e72-4366-8946-d808e2e3d465'

In [5]:
ps3 = PySage3(conf, prod_type)
cb = ps3.rooms[room_id].boards[board_id]

Configuring ps3 client ... 
Completed configuring Sage3 Client


In [8]:
ps3.get_apps(room_id, board_id, add_tags=True, filter_tags= keys_to_remove)

{'7bb2d8d9-c937-4a8f-bf6f-d31cf34018cd': {'data': {'title': 'A title',
   'position': {'x': 87, 'y': 243, 'z': 0},
   'size': {'width': 200, 'height': 200, 'depth': 0},
   'type': 'Stickie',
   'state': {'text': 'hellow world', 'color': 'red', 'fontSize': 22}},
  'tags': []},
 '01010d5c-0b4c-4281-8745-745f89cf7210': {'data': {'title': 'Mahdi Bel-caid',
   'position': {'x': 646, 'y': 380, 'z': 0},
   'size': {'width': 400, 'height': 420, 'depth': 0},
   'type': 'Stickie',
   'state': {'text': 'My Test Stickie', 'fontSize': 36, 'color': 'yellow'}},
  'tags': []}}

In [9]:
llm_4o = ChatOpenAI(temperature=0.1, model="gpt-4o")
# llm_4o.invoke("hello")

In [ ]:
# example_prompt = PromptLoader.create_prompt(PROMPTS_DIR, "few_shot", "reframe_select_apps")
# q = 'Can you delete all the red stickies with a greeting or hello'
# reframe_select_chain =  example_prompt | llm_4o
# reframe_select_chain.invoke({"query":q})

In [ ]:
# example_prompt = PromptLoader.create_prompt(PROMPTS_DIR, "few_shot", "select_apps")

# q = 'Can you delete all the red stickies with a greeting'
# all_apps = ps3.get_apps(room_id, board_id, add_tags=True, filter_tags= keys_to_remove)
# prompt_text = example_prompt.format(originalItems=all_apps, query=q)
# prompt_text [0:100]+ " ...."


In [ ]:
# (example_prompt | llm_4o).invoke({"originalItems":all_apps, "query":q})

In [ ]:
pwd

In [ ]:
create_app_prompt = PromptLoader.create_prompt(PROMPTS_DIR, "zero_shot", "create_app")

q = 'Can you create a new Stickie that says "here you go"'
all_apps = ps3.get_apps(room_id, board_id, add_tags=True, filter_tags= keys_to_remove)
prompt_text = create_app_prompt.format(originalItems=all_apps, query=q)
print(prompt_text [0:100]+ " ....")


In [ ]:
from src.tools.wall_tools import CreateAppTool
tools = {"CreateAppTool": CreateAppTool(ps3)}
llm_chain = llm_4o.bind_tools([tools["CreateAppTool"]])

In [ ]:
resp = llm_chain.invoke("create a red stickie with the text hello world")
resp

In [ ]:
resp.tool_calls

In [ ]:
from src.smartbits import SmartBit
sb = SmartBit(**resp.tool_calls[0]['args']['smartbit'])
sb


In [ ]:
tool_name = resp.tool_calls[0]["name"]
tool = tools[tool_name]
tool.invoke({"smartbit":  sb})

In [ ]:
from src.tools.wall_tools import DeleteAppTool
tools = {"DeleteAppTool": DeleteAppTool(ps3)}
llm = llm_4o.bind_tools([tools["DeleteAppTool"]])

In [ ]:
current_apps = ps3.get_apps(room_id, board_id, add_tags=True, filter_tags= keys_to_remove)
current_apps

In [ ]:
delete_prompt = PromptLoader.create_prompt(PROMPTS_DIR, "zero_shot", "delete_app")
delete_prompt

In [ ]:
query = "Delete all the stickies with the text hellow world"
resp  = (delete_prompt | llm).invoke({"originalItems":current_apps, "query": query})
resp

In [ ]:
resp.tool_calls

In [ ]:
tool_name = resp.tool_calls[0]['name']
app_id = resp.tool_calls[0]['args']['app_id']

In [ ]:
tools[tool_name].invoke({"app_id": app_id})

In [ ]:
ps3.get_apps(add_tags=True)

In [ ]:
current_apps = ps3.get_apps(room_id, board_id, add_tags=True, filter_tags= keys_to_remove)
current_apps

In [ ]:
query = "Delete all the mccormick recsys pdf file"
resp  = (delete_prompt | llm).invoke({"originalItems":current_apps, "query": query})
resp

In [ ]:
resp.tool_calls

In [ ]:
ps3.get_apps(filter_tags= keys_to_remove_2)

In [ ]:
all_apps = ps3.get_apps(room_id, board_id, add_tags=True, filter_tags= keys_to_remove)
all_apps


In [ ]:
ps3.get_apps_text(all_apps)

In [ ]:
all_apps

In [ ]:
app = all_apps['ca50b398-86d2-4518-8b67-6a426907d61a']
app

In [ ]:
app_type = app.get('data', {}).get('type', None)
app_type

In [ ]:
text = ""
text += app.get('data', {}).get('state', {}).get('text', "")+"\n"
text

In [ ]:
app

In [ ]:
asset_id = app.get('data', {}).get('state', {}).get('assetid')
asset_id

In [ ]:
ps3.list_assets(asset_id=asset_id)

In [ ]:
ps3.s3_comm.get_pdf_text('dist/apps/homebase/assets/92d09434-3607-4dd8-9978-9111190b1873.pdf', pages=[0])

In [ ]:

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
output_parser = StrOutputParser()

chain = prompt | llm_4o | output_parser

chain.invoke({"topic": "ice cream"})

In [ ]:
import importlib
import src
importlib.reload(src.chains.chain_factory)
importlib.reload(src.tools.wall_tools)
from src.chains.chain_factory import ChainFactory
from langchain_core.output_parsers import StrOutputParser

In [ ]:
cf = ChainFactory()
joke_chain = cf.create_chain("sumarize", "zero_shot/joke", llm_4o, StrOutputParser)
joke_chain.invoke({"topic": "ice cream"})

In [ ]:
cf.chains['jokes'].invoke({"topic": "ice cream"})

In [ ]:
cf.create_chain("summarize_text", "zero_shot/summarize_text", llm_4o, StrOutputParser)

In [ ]:
text = ps3.s3_comm.get_pdf_text('dist/apps/homebase/assets/92d09434-3607-4dd8-9978-9111190b1873.pdf', pages=[0])
cf.chains['summarize_text'].invoke({"query": text})

In [8]:
wall_agent_prompt = ChatPromptTemplate.from_messages(
    [  
        (
            "system",
            "You are a basic SAGE3 subagent and you can create or delete apps (also called widgets) on the wall, like Stickies, Counter or PDFViewers. "
            "You can also access and interact with the text contained in the apps on the wall to answer questions of provide summary but "
			"you are not supposed to be interactive. Try to execute the code without prompting the user for more info.\n"
			"\nCurrent Apps on the wall: \n{current_apps}\n\n"
            "If none of your tools are appropriate for it, then \"CompleteOrEscalate\" the dialog to the host assistant. "
            "Do not waste the user's time. Do not create fictitious tools or functions.",
        ),
        ("placeholder", "{messages}"),
    ]
).partial()

In [9]:
wall_agent_prompt.invoke({"current_apps":"abc", "messages":[("human", "Query: create a stickie with the text hello world")]}).messages

[chain/start] [prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "current_apps": "abc",
  "messages": [
    [
      "human",
      "Query: create a stickie with the text hello world"
    ]
  ]
}
[chain/end] [prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]


[SystemMessage(content='You are a basic SAGE3 subagent and you can create or delete apps (also called widgets) on the wall, like Stickies, Counter or PDFViewers. You can also access and interact with the text contained in the apps on the wall to answer questions of provide summary but you are not supposed to be interactive. Try to execute the code without prompting the user for more info.\n\nCurrent Apps on the wall: \nabc\n\nIf none of your tools are appropriate for it, then "CompleteOrEscalate" the dialog to the host assistant. Do not waste the user\'s time. Do not create fictitious tools or functions.'),
 HumanMessage(content='Query: create a stickie with the text hello world')]

In [21]:
from src.tools.wall_tools import CreateAppTool, DeleteAppTool, SummarizeAppsTool, CompleteOrEscalate
wall_agent_safe_tools = [CreateAppTool(ps3), SummarizeAppsTool(ps3)]
wall_agent_sensitive_tools = [DeleteAppTool(ps3)]
wall_agent_tools = wall_agent_safe_tools + wall_agent_sensitive_tools
wall_agent_runnable = wall_agent_prompt | llm_4o.bind_tools(
    wall_agent_tools + [CompleteOrEscalate]
)

In [22]:
from langchain.globals import set_debug
set_debug(True)

wall_agent_runnable.invoke({"current_apps" :"abc", "messages": [("human", "Query: create a stickie with the text hello world")]})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "current_apps": "abc",
  "messages": [
    [
      "human",
      "Query: create a stickie with the text hello world"
    ]
  ]
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "current_apps": "abc",
  "messages": [
    [
      "human",
      "Query: create a stickie with the text hello world"
    ]
  ]
}
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: You are a basic SAGE3 subagent and you can create or delete apps (also called widgets) on the wall, like Stickies, Counter or PDFViewers. You can also access and interact with the text contained in the apps on the wall to answer questions of provide summary but you are not supposed to be interactive. Try to execute the code without prompt

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Ub9myh4huD6dQhfcwdjwrZ8K', 'function': {'arguments': '{"smartbit":{"data":{"position":{"x":100,"y":100,"z":0},"size":{"width":200,"height":200,"depth":0},"type":"Stickie"},"state":{"text":"hello world","color":"yellow","fontSize":22}}}', 'name': 'CreateAppTool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 951, 'total_tokens': 1018}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6e8d2430-8abd-4c4a-8c0e-3b8c2782ca0f-0', tool_calls=[{'name': 'CreateAppTool', 'args': {'smartbit': {'data': {'position': {'x': 100, 'y': 100, 'z': 0}, 'size': {'width': 200, 'height': 200, 'depth': 0}, 'type': 'Stickie'}, 'state': {'text': 'hello world', 'color': 'yellow', 'fontSize': 22}}}, 'id': 'call_Ub9myh4huD6dQhfcwdjwrZ8K'}])

In [23]:
def handle_tool_error(state) -> dict:
    error = state.get("error")
    tool_calls = state["messages"][-1].tool_calls
    return {
        "messages": [
            ToolMessage(
                content=f"Error: {repr(error)}\n please fix your mistakes.",
                tool_call_id=tc["id"],
            )
            for tc in tool_calls
        ]
    }

def wall_agent_routes(
    state: State,
) -> Literal[
    "wall_agent_safe_tools",
    "wall_agent_sensitive_tools",
    # "complete_or_escalate",
    "__end__",
]:
    last_msg = state["messages"][-1]
    if not last_msg.tool_calls:
        return "__end__"

    tool_calls = state["messages"][-1].tool_calls
    # did_cancel = any(tc["name"] == CompleteOrEscalate.__name__ for tc in tool_calls)
    # if did_cancel:
    #     return "complete_or_escalate"
    
    safe_toolnames = [t.name for t in wall_agent_safe_tools]
    if all(tc["name"] in safe_toolnames for tc in tool_calls):
        return "wall_agent_safe_tools"
    return "wall_agent_sensitive_tools"


from langchain_core.runnables import RunnableLambda
def create_tool_node_with_fallback(tools: list) -> dict:
    return ToolNode(tools).with_fallbacks(
        [RunnableLambda(handle_tool_error)], exception_key="error"
    )


In [87]:
from typing import Annotated, Optional, Literal, Dict
from langchain.pydantic_v1 import BaseModel, Field, UUID4

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.runnables import Runnable, RunnableConfig

class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]
    user_id: Optional[UUID4]
    current_apps: Optional[Dict]


class Assistant:
    def __init__(self, runnable: Runnable):
        self.runnable = runnable

    def __call__(self, state: State, config: RunnableConfig):
        print("-----START State-----")
        print(state)
        print("-----END State-----")
        print("-----START Model Invocation-----")
        result = self.runnable.invoke(state)
        print("-----END Model Invocation-----")
        
        return {"messages": result}
        
    
graph_builder = StateGraph(State)



In [88]:
from langgraph.checkpoint.sqlite import SqliteSaver
import uuid
graph_builder.add_node("wall_agent", Assistant(wall_agent_runnable))
graph_builder.add_node("wall_agent_safe_tools", create_tool_node_with_fallback(wall_agent_safe_tools))
graph_builder.add_node("wall_agent_sensitive_tools", create_tool_node_with_fallback(wall_agent_sensitive_tools))

graph_builder.add_conditional_edges("wall_agent", wall_agent_routes)
graph_builder.add_edge("wall_agent_safe_tools", "wall_agent")
graph_builder.add_edge("wall_agent_sensitive_tools", "wall_agent")

graph_builder.set_entry_point("wall_agent")
memory = SqliteSaver.from_conn_string(":memory:")
graph = graph_builder.compile(checkpointer=memory)
thread_id = str(uuid.uuid4())

config = {
    "configurable": {
        # Checkpoints are accessed by thread_id
        "thread_id": thread_id,
    }
}

In [66]:
graph.invoke({"messages": [("human", "Create a red stickie with the text hellow world")]}, config)

[chain/start] [chain:LangGraph] Entering Chain run with input:
{
  "messages": [
    [
      "human",
      "Create a red stickie with the text hellow world"
    ]
  ]
}
[chain/start] [chain:LangGraph > chain:__start__] Entering Chain run with input:
{
  "messages": [
    [
      "human",
      "Create a red stickie with the text hellow world"
    ]
  ]
}
[chain/end] [chain:LangGraph > chain:__start__] s] Exiting Chain run with output:
{
  "messages": [
    [
      "human",
      "Create a red stickie with the text hellow world"
    ]
  ]
}
[chain/start] [chain:LangGraph > chain:wall_agent] Entering Chain run with input:
[inputs]
-----START State-----
{'messages': [HumanMessage(content='Create a red stickie with the text hellow world', id='668e1a97-67f8-4a00-a84b-c4a68d8871de')], 'user_id': None, 'current_apps': None}
-----END State-----
-----START Model Invocation-----
[chain/start] [chain:LangGraph > chain:wall_agent > chain:RunnableSequence] Entering Chain run with input:
[inputs]
[

{'messages': [HumanMessage(content='Create a red stickie with the text hellow world', id='668e1a97-67f8-4a00-a84b-c4a68d8871de'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pa2uf32kfboyCkbXIVKhWSqD', 'function': {'arguments': '{"smartbit":{"data":{"position":{"x":100,"y":100,"z":0},"size":{"width":200,"height":200,"depth":0},"type":"Stickie"},"state":{"text":"hellow world","color":"red","fontSize":22}}}', 'name': 'CreateAppTool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 951, 'total_tokens': 1019}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-61bdace7-a328-4bf2-b551-cff20a7334fc-0', tool_calls=[{'name': 'CreateAppTool', 'args': {'smartbit': {'data': {'position': {'x': 100, 'y': 100, 'z': 0}, 'size': {'width': 200, 'height': 200, 'depth': 0}, 'type': 'Stickie'}, 'state': {'text': 'hellow world', 'color': 'red', 'fontSize': 22}

In [ ]:
# resp = graph.nodes['wall_agent'].invoke({"messages": [("human", "Create a red stickie with the text hellow world")], 'current_apps':'abc'})
# resp

In [33]:
resp = llm_4o.bind_tools([CreateAppTool(ps3), DeleteAppTool(ps3)]).invoke("Create three red stickies with the text hellow world and delete all the yellow the stickies on the wall")



[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Create three red stickies with the text hellow world and delete all the yellow the stickies on the wall"
  ]
}
[llm/end] [llm:ChatOpenAI] [3.69s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "",
        "generation_info": {
          "finish_reason": "tool_calls",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "",
            "additional_kwargs": {
              "tool_calls": [
                {
                  "id": "call_hAYYS7JmdUvB0zHG1cSQsBKj",
                  "function": {
                    "arguments": "{\"smartbit\": {\"data\": {\"position\": {\"x\": 100, \"y\": 100, \"z\": 0}, \"size\": 

In [34]:
resp.tool_calls

[{'name': 'CreateAppTool',
  'args': {'smartbit': {'data': {'position': {'x': 100, 'y': 100, 'z': 0},
     'size': {'width': 200, 'height': 200, 'depth': 0},
     'type': 'Stickie'},
    'state': {'text': 'hellow world', 'color': 'red', 'fontSize': 22}}},
  'id': 'call_hAYYS7JmdUvB0zHG1cSQsBKj'},
 {'name': 'CreateAppTool',
  'args': {'smartbit': {'data': {'position': {'x': 400, 'y': 100, 'z': 0},
     'size': {'width': 200, 'height': 200, 'depth': 0},
     'type': 'Stickie'},
    'state': {'text': 'hellow world', 'color': 'red', 'fontSize': 22}}},
  'id': 'call_xCz90KuXiSldkWumgdce6ayS'},
 {'name': 'CreateAppTool',
  'args': {'smartbit': {'data': {'position': {'x': 700, 'y': 100, 'z': 0},
     'size': {'width': 200, 'height': 200, 'depth': 0},
     'type': 'Stickie'},
    'state': {'text': 'hellow world', 'color': 'red', 'fontSize': 22}}},
  'id': 'call_3YDcuHsIGbYd5djAh004eq2i'}]

In [31]:
tn = ToolNode([CreateAppTool(ps3)])

In [32]:
tn.invoke({"messages": [resp]})

[tool/start] [tool:CreateAppTool] Entering Tool run with input:
"{'smartbit': {'data': {'position': {'x': 0, 'y': 0, 'z': 0}, 'size': {'width': 200, 'height': 200, 'depth': 0}, 'type': 'Stickie'}, 'state': {'text': 'hellow world', 'color': 'red', 'fontSize': 22}}}"
[tool/start] [tool:CreateAppTool] Entering Tool run with input:
"{'smartbit': {'data': {'position': {'x': 220, 'y': 0, 'z': 0}, 'size': {'width': 200, 'height': 200, 'depth': 0}, 'type': 'Stickie'}, 'state': {'text': 'hellow world', 'color': 'red', 'fontSize': 22}}}"
[tool/start] [tool:CreateAppTool] Entering Tool run with input:
"{'smartbit': {'data': {'position': {'x': 440, 'y': 0, 'z': 0}, 'size': {'width': 200, 'height': 200, 'depth': 0}, 'type': 'Stickie'}, 'state': {'text': 'hellow world', 'color': 'red', 'fontSize': 22}}}"
Running Create app
[tool/end] [tool:CreateAppTool] s] Exiting Tool run with output:
"Done with CreateAppTool"
Running Create app
[tool/end] [tool:CreateAppTool] s] Exiting Tool run with output:
"Don

{'messages': [ToolMessage(content='Done with CreateAppTool', name='CreateAppTool', tool_call_id='call_XZT2hQsZc35GpFxId7cJyFgP'),
  ToolMessage(content='Done with CreateAppTool', name='CreateAppTool', tool_call_id='call_qF5VS4HT6o7Y2lGpEvaU5fuS'),
  ToolMessage(content='Done with CreateAppTool', name='CreateAppTool', tool_call_id='call_b7LusNN0mwSKqYoLKPSdhE2b')]}

In [ ]:
!pip install langgraph

In [ ]:
from typing import Annotated, Optional, Literal, Dict
from langchain.pydantic_v1 import BaseModel, Field, UUID4

from typing_extensions import TypedDict

from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]
    user_id: Optional[UUID4]
    current_apps: Dict


graph_builder = StateGraph(State)

def node_1(state: State):
    return {"messages": [("human", "node_1 input")]}

# def node_2(state: State):
#     return {"messages": [("human", "how are you")]}

node_2_p = ChatPromptTemplate.from_messages(
    [   ("human", "node_2 before messages"),  
        ("placeholder", "{messages}"),
     ("human", "node_2 after messages"),  
     
    ]
).partial()

node_2_runnable = node_2_p | llm_4o

graph_builder.add_node("node_1", node_1)
graph_builder.add_node("node_2", Assistant(node_2_runnable))
graph_builder.add_edge("node_1", "node_2")

graph_builder.set_entry_point("node_1")
graph_builder.set_finish_point("node_2")
graph = graph_builder.compile()

In [ ]:
# from IPython.display import Image, display

# try:
#     display(Image(graph.get_graph().draw_mermaid_png()))
# except:
#     # This requires some extra dependencies and is optional
#     pass

In [ ]:
from langchain_core.messages.human import HumanMessage

In [ ]:
s = {'messages': [HumanMessage(content='Hello Input', id='cba94913-770e-4a33-877b-b4ad0e533550'), HumanMessage(content='node_1 input', id='3b7df344-7495-491d-b358-eb6013dd09b3')]}

In [ ]:
s_2 = node_2_runnable.first.invoke(s)

In [ ]:
node_2_runnable.last.invoke(s_2)

In [ ]:
from langchain_core.messages.ai import AIMessage
result = AIMessage(content="It seems like you're trying to outline a sequence or structure involving nodes and messages. However, the context is a bit unclear. Could you please provide more details or clarify what you're trying to achieve? Are you working on a specific project or problem involving nodes and messages?", response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 35, 'total_tokens': 88}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None}, id='run-69c51c62-7b79-45dd-b017-0ca38bc20363-0')

In [ ]:
isinstance(result.content, list)

In [ ]:
tools_condition({"messages": [resp]})

In [ ]:
resp.tool_calls

In [89]:
ms = [("human", "Create a red stickie with the text hellow world")]

In [90]:
graph.get_state(config).next

()

In [91]:
resp = graph.nodes['wall_agent'].invoke({"messages": ms, 'current_apps':"abc"})
resp


-----START State-----
{'messages': [('human', 'Create a red stickie with the text hellow world')], 'current_apps': 'abc'}
-----END State-----
-----START Model Invocation-----
[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "messages": [
    [
      "human",
      "Create a red stickie with the text hellow world"
    ]
  ],
  "current_apps": "abc"
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "messages": [
    [
      "human",
      "Create a red stickie with the text hellow world"
    ]
  ],
  "current_apps": "abc"
}
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: You are a basic SAGE3 subagent and you can create or delete apps (also called widgets) on the wall, like Stickies, Counter or PDFViewers. You can also access and interac

{'messages': AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pqcQfY2m1bybvQSn7KMAaA7u', 'function': {'arguments': '{"smartbit":{"data":{"position":{"x":0,"y":0,"z":0},"size":{"width":200,"height":200,"depth":0},"type":"Stickie"},"state":{"text":"hellow world","color":"red","fontSize":22}}}', 'name': 'CreateAppTool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 951, 'total_tokens': 1019}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1e3d65e9-b6b3-464e-8c20-524f111cb192-0', tool_calls=[{'name': 'CreateAppTool', 'args': {'smartbit': {'data': {'position': {'x': 0, 'y': 0, 'z': 0}, 'size': {'width': 200, 'height': 200, 'depth': 0}, 'type': 'Stickie'}, 'state': {'text': 'hellow world', 'color': 'red', 'fontSize': 22}}}, 'id': 'call_pqcQfY2m1bybvQSn7KMAaA7u'}])}

In [92]:
resp

{'messages': AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pqcQfY2m1bybvQSn7KMAaA7u', 'function': {'arguments': '{"smartbit":{"data":{"position":{"x":0,"y":0,"z":0},"size":{"width":200,"height":200,"depth":0},"type":"Stickie"},"state":{"text":"hellow world","color":"red","fontSize":22}}}', 'name': 'CreateAppTool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 951, 'total_tokens': 1019}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1e3d65e9-b6b3-464e-8c20-524f111cb192-0', tool_calls=[{'name': 'CreateAppTool', 'args': {'smartbit': {'data': {'position': {'x': 0, 'y': 0, 'z': 0}, 'size': {'width': 200, 'height': 200, 'depth': 0}, 'type': 'Stickie'}, 'state': {'text': 'hellow world', 'color': 'red', 'fontSize': 22}}}, 'id': 'call_pqcQfY2m1bybvQSn7KMAaA7u'}])}

In [93]:
graph.update_state(config, resp, as_node="wall_agent")

[chain/start] [chain:LangGraphUpdateState] Entering Chain run with input:
[inputs]
[chain/start] [chain:LangGraphUpdateState > chain:ChannelWrite<wall_agent,messages,user_id,current_apps>] Entering Chain run with input:
[inputs]
[chain/end] [chain:LangGraphUpdateState > chain:ChannelWrite<wall_agent,messages,user_id,current_apps>] s] Exiting Chain run with output:
[outputs]
[chain/start] [chain:LangGraphUpdateState > chain:wall_agent_routes] Entering Chain run with input:
[inputs]
[chain/end] [chain:LangGraphUpdateState > chain:wall_agent_routes] s] Exiting Chain run with output:
{
  "output": "wall_agent_safe_tools"
}
[chain/start] [chain:LangGraphUpdateState > chain:ChannelWrite<branch:wall_agent:wall_agent_routes:wall_agent_safe_tools>] Entering Chain run with input:
[inputs]
[chain/end] [chain:LangGraphUpdateState > chain:ChannelWrite<branch:wall_agent:wall_agent_routes:wall_agent_safe_tools>] s] Exiting Chain run with output:
[outputs]
[chain/end] [chain:LangGraphUpdateState] s] E

{'configurable': {'thread_id': '8926e059-65e4-4190-a4ac-a1f5db8f37c4',
  'thread_ts': '1ef25722-c722-6ce8-bfff-e13ff84ed45c'}}

In [94]:
graph.get_state(config).next

('wall_agent_safe_tools',)

In [96]:
ms += [resp['messages']]
ms

2

In [97]:
resp = graph.nodes['wall_agent_safe_tools'].invoke({"messages": ms, 'current_apps':"abc"})
resp


[chain/start] [chain:RunnableWithFallbacks] Entering Chain run with input:
[inputs]
[tool/start] [chain:RunnableWithFallbacks > tool:CreateAppTool] Entering Tool run with input:
"{'smartbit': {'data': {'position': {'x': 0, 'y': 0, 'z': 0}, 'size': {'width': 200, 'height': 200, 'depth': 0}, 'type': 'Stickie'}, 'state': {'text': 'hellow world', 'color': 'red', 'fontSize': 22}}}"
Running Create app
[tool/end] [chain:RunnableWithFallbacks > tool:CreateAppTool] s] Exiting Tool run with output:
"Done with CreateAppTool"
[chain/end] [chain:RunnableWithFallbacks] s] Exiting Chain run with output:
[outputs]


{'messages': [ToolMessage(content='Done with CreateAppTool', name='CreateAppTool', tool_call_id='call_pqcQfY2m1bybvQSn7KMAaA7u')]}

In [98]:
resp

{'messages': [ToolMessage(content='Done with CreateAppTool', name='CreateAppTool', tool_call_id='call_pqcQfY2m1bybvQSn7KMAaA7u')]}

In [99]:
graph.update_state(config, resp, as_node="wall_agent_safe_tools")
ms += [resp['messages']]
ms


[chain/start] [chain:LangGraphUpdateState] Entering Chain run with input:
[inputs]
[chain/end] [chain:LangGraphUpdateState] s] Exiting Chain run with output:
[outputs]


[('human', 'Create a red stickie with the text hellow world'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pqcQfY2m1bybvQSn7KMAaA7u', 'function': {'arguments': '{"smartbit":{"data":{"position":{"x":0,"y":0,"z":0},"size":{"width":200,"height":200,"depth":0},"type":"Stickie"},"state":{"text":"hellow world","color":"red","fontSize":22}}}', 'name': 'CreateAppTool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 951, 'total_tokens': 1019}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1e3d65e9-b6b3-464e-8c20-524f111cb192-0', tool_calls=[{'name': 'CreateAppTool', 'args': {'smartbit': {'data': {'position': {'x': 0, 'y': 0, 'z': 0}, 'size': {'width': 200, 'height': 200, 'depth': 0}, 'type': 'Stickie'}, 'state': {'text': 'hellow world', 'color': 'red', 'fontSize': 22}}}, 'id': 'call_pqcQfY2m1bybvQSn7KMAaA7u'}]),
 [ToolMessage(content='Done wi

In [100]:
graph.get_state(config).next

('wall_agent',)

In [74]:
n =  ToolNode([CreateAppTool(ps3)])
resp_t = n.invoke({"messages": [resp['messages']]})

[tool/start] [tool:CreateAppTool] Entering Tool run with input:
"{'smartbit': {'data': {'position': {'x': 100, 'y': 100, 'z': 0}, 'size': {'width': 200, 'height': 200, 'depth': 0}, 'type': 'Stickie'}, 'state': {'text': 'hellow world', 'color': 'red', 'fontSize': 22}}}"
Running Create app
[tool/end] [tool:CreateAppTool] s] Exiting Tool run with output:
"Done with CreateAppTool"


In [75]:
ms += resp_t['messages']
ms

[('human', 'Create a red stickie with the text hellow world'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_KSrcMaPkoxmg8dtWoD5E9HIt', 'function': {'arguments': '{"smartbit":{"data":{"position":{"x":100,"y":100,"z":0},"size":{"width":200,"height":200,"depth":0},"type":"Stickie"},"state":{"text":"hellow world","color":"red","fontSize":22}}}', 'name': 'CreateAppTool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 951, 'total_tokens': 1019}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b5c9fce1-22a9-4697-94bc-f72ebe754151-0', tool_calls=[{'name': 'CreateAppTool', 'args': {'smartbit': {'data': {'position': {'x': 100, 'y': 100, 'z': 0}, 'size': {'width': 200, 'height': 200, 'depth': 0}, 'type': 'Stickie'}, 'state': {'text': 'hellow world', 'color': 'red', 'fontSize': 22}}}, 'id': 'call_KSrcMaPkoxmg8dtWoD5E9HIt'}]),
 ToolMessage(content='

In [51]:
graph.get_state()

ValueError: No checkpointer set